In [ ]:
import math
import copy
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Get Data
train_raw = np.genfromtxt('./ass3_data/poker-hand-training-true.data', delimiter=',')
test_raw = np.genfromtxt('./ass3_data/poker-hand-testing.data', delimiter=',')

In [ ]:
# One Hot encoding for PART A
train_df = pd.DataFrame(np.delete(train_raw, -1, 1))
test_df = pd.DataFrame(np.delete(train_raw, -1, 1))
train_outraw = train_raw[:, -1]
test_outraw = test_raw[:, -1]

train_data = pd.get_dummies(train_df, columns=train_df.columns).values
test_data = pd.get_dummies(test_df, columns=train_df.columns).values
train_output = pd.get_dummies(train_outraw).values
test_output = pd.get_dummies(test_outraw).values

num_features = len(train_data[0])

In [ ]:
def sigmoid(x):
    return 1/(1 + math.exp(-x))

In [ ]:
debug_nn = False

class Neural_Net:
    batch_size = 0
    num_inputs = 0
    layers = []
    num_outputs = 0
    learning_rate = 0.1
    
    def __init__(self, b, i, h, o):
        self.batch_size = b
        self.num_inputs = i
        self.num_outputs = o
        self.layers = []
        self.learning_rate = 0.1
        
        # Initialize Parameters for Hidden Layers
        for layer in range(len(h)):
            units = h[layer]
            layer_weights = []
            for unit in range(units):
                unit_weights = []
                unit_weights.append(random.random()-0.5) # Add bias weight
                if layer == 0:
                    for inp in range(self.num_inputs):
                        unit_weights.append(random.random()-0.5)
                else:
                    for inp in range(h[layer-1]):
                        unit_weights.append(random.random()-0.5)
                layer_weights.append(np.array(unit_weights))
            self.layers.append(np.array(layer_weights))
        self_layers = np.array(self.layers)
            
        # Output Layer Perceptrons
        layer_weights = []
        for unit in range(self.num_outputs):
            unit_weights = []
            unit_weights.append(random.random()-0.5) # Add bias weight
            for inp in range(h[-1]):
                unit_weights.append(random.random()-0.5)
            layer_weights.append(np.array(unit_weights))
        self.layers.append(np.array(layer_weights))
        print ("Layer weights: {0}".format(self.layers))
        
    def train(self, dataset, target):
        batch = []
        batch_target = []
        num_epochs = 5000
        prev_loss = 99999999
        for epoch in range(num_epochs):
            loss_func = 0
            for s_index in range(len(dataset)):
                sample = dataset[s_index]
                batch.append(sample)
                batch_target.append(target[s_index])
                if len(batch) < self.batch_size and s_index != len(dataset)-1:
                    continue

                if debug_nn:
                    print ("Starting with batch {0}".format(batch))
                netj_all = []
                xjk_all = []
                # Forward Propagation
                for batch_sample in batch:
                    xjk = copy.deepcopy(batch_sample)
                    # Hidden Layers Computation
                    for l_index in range(len(self.layers)):
                        if debug_nn:
                            print ("Input to layer {0}: {1}".format(l_index, xjk))
                            layer_weights = self.layers[l_index]
                            print ("Layer {0} weights: {1}".format(l_index, layer_weights))
                        else:
                            layer_weights = self.layers[l_index]
                        if l_index < len(netj_all):
                            xjk_complete = np.append(xjk, [1])
                            xjk_all += (np.array([xjk_complete]))
                            netj_all[l_index] += (np.dot(np.delete(layer_weights, -1, 1), xjk) + layer_weights[:, -1])
                        else:
                            xjk_complete = np.append(xjk, [1])
                            xjk_all.append(np.array([xjk_complete]))
                            netj_all.append(np.dot(np.delete(layer_weights, -1, 1), xjk) + layer_weights[:, -1])
                        xjk = np.array(list(map(sigmoid, netj_all[-1])))
                    output = xjk
                    
                if debug_nn:
                    print ("Forward Feed complete.")
                    print ("Output: {0}".format(output))
                    print ("Net J: {0}".format(netj_all))
                    print ("X: {0}".format(xjk_all))

                # Back Propagation
                updated_weights = copy.deepcopy(self.layers)
                if debug_nn:
                    print ("Back Proping for output layer")
                    layer_weights = self.layers[-1]
                    print ("Working on layer weights: {0}".format(layer_weights))
                else:
                    layer_weights = self.layers[-1]
                xjk = xjk_all[-1]
                del_netj = []
                for sample_target in batch_target:
                    loss_func += np.sum((sample_target-output)*(sample_target-output))
                    if del_netj:
                        del_netj += -1*(sample_target-output)*output*(1-output)
                    else:
                        del_netj = -1*(sample_target-output)*output*(1-output)
                if debug_nn:
                    print ("Del netj at output layer: {0}".format(del_netj))
                    print ("X at output layer: {0}".format(xjk))
                    del_j = np.matmul(np.array([del_netj]).T, xjk)
                    print ("Output derivative: {0}".format(del_j))
                    print ("Subtracting {0} - {1}".format(layer_weights, self.learning_rate*del_j/self.batch_size))
                    updated_weights[-1] = np.subtract(layer_weights, self.learning_rate*del_j/self.batch_size)
                    print ("Updated weights for Output Layer: {0}".format(updated_weights[-1]))
                else:
                    del_j = np.matmul(np.array([del_netj]).T, xjk)
                    updated_weights[-1] = np.subtract(layer_weights, self.learning_rate*del_j/self.batch_size)
                    
                # Hidden Layers
                for layer in reversed(range(len(self.layers[:-1]))):  
                    if debug_nn:
                        print ("Back proping for layer {0}".format(layer))
                        layer_weights = self.layers[layer]
                        print ("Working on layer weights: {0}".format(self.layers[layer]))
                    else:
                        layer_weights = self.layers[layer]
                    netj = netj_all[layer]
                    xjk = xjk_all[layer]
                    signetj = np.array(list(map(sigmoid,netj)))
                    del_lj = (np.delete(self.layers[layer+1], -1, 1) * signetj*(1-signetj))
                    if debug_nn:
                        print ("Del lj at layer {0}: {1}".format(layer, del_lj))
                        del_netj = np.matmul(del_lj.T, del_netj)
                        print ("Del netj at layer {0}: {1}".format(layer, del_netj))
                        print ("X at output layer: {0}".format(xjk))
                        del_j = np.matmul(np.array([del_netj]).T, xjk)
                        print ("Final derivative: {0}".format(del_j))
                    else:
                        del_netj = np.matmul(del_lj.T, del_netj)
                        del_j = np.matmul(np.array([del_netj]).T, xjk)
                        
                    updated_weights[layer] = np.subtract(layer_weights, self.learning_rate*del_j/self.batch_size)
                self.layers = updated_weights

                if debug_nn:
                    print ("Updated weights: {0}".format(self.layers))
                    print ("-----------------------------------------")
                batch = []
                batch_target = []
            
            if abs(prev_loss-loss_func) < 0.00005:
                break
            prev_loss = loss_func
            if debug_nn:
                print (loss_func)
        print ("Num Epochs: {0}".format(epoch))
        print ("Final Weights: {0}".format(self.layers))
        
    def predict(self, sample):
        xjk = copy.deepcopy(sample)
        # Hidden Layers Computation
        netj_all = []
        for l_index in range(len(self.layers)):
            print ("Input to layer {0}: {1}".format(l_index, xjk))
            layer_weights = self.layers[l_index]
            print ("Layer {0} weights: {1}".format(l_index, layer_weights))
            if l_index < len(netj_all):
                netj_all[l_index] += (np.dot(np.delete(layer_weights, -1, 1), xjk) + layer_weights[:, -1])
            else:
                netj_all.append(np.dot(np.delete(layer_weights, -1, 1), xjk) + layer_weights[:, -1])
            xjk = np.array(list(map(sigmoid, netj_all[-1])))
        output = xjk
        print (output)

In [ ]:
nn = Neural_Net(1, 3, [1], 2)
# nn = Neural_Net(1, num_features, [5], 10)

In [ ]:
# dataset = [[0,0], [0,1], [1,0], [1,1]]
# target = [[1,0], [0,1], [0,1], [0,1]]
dataset = [[0,0,1], [0,1,1], [1,0,1], [1,1,1], [0,0,0], [1,0,0]]
target = [[1,0], [0,1], [0,1], [0,1], [1,0], [1,0]]
nn.train(dataset, target)
# m = 1000
# nn.train(train_data[0:m], train_output[0:m])

In [ ]:
nn.predict([1,1,0])

In [ ]:
sigmoid(-0.67692362)

In [ ]:
a = np.array([2,3])
b = np.array([1,1])
print (np.sum((a-b)*(a-b)))